In [1]:
! pip install datasets transformers

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 80 not upgraded.


In [4]:
import transformers

print(transformers.__version__)

4.42.3


In [5]:
from transformers.utils import send_example_telemetry

send_example_telemetry("question_answering_notebook", framework="pytorch")

In [6]:
from datasets import load_dataset, load_metric

In [7]:
from datasets import load_dataset

# Load the PIQA dataset
dataset = load_dataset("piqa")

The repository for piqa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/piqa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/16113 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3084 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1838 [00:00<?, ? examples/s]

In [8]:
# Display the keys of the dataset
print(dataset)

# Display the first example from the training set
first_example = dataset['train'][0]
print(first_example)

DatasetDict({
    train: Dataset({
        features: ['goal', 'sol1', 'sol2', 'label'],
        num_rows: 16113
    })
    test: Dataset({
        features: ['goal', 'sol1', 'sol2', 'label'],
        num_rows: 3084
    })
    validation: Dataset({
        features: ['goal', 'sol1', 'sol2', 'label'],
        num_rows: 1838
    })
})
{'goal': "When boiling butter, when it's ready, you can", 'sol1': 'Pour it onto a plate', 'sol2': 'Pour it into a jar', 'label': 1}


In [9]:
train_dataset = dataset['train']
val_dataset = dataset['validation']
test_dataset = dataset['test']

In [10]:
print(train_dataset[0])  # This will show you the structure of a sample in the train split
print(val_dataset[0])
print(test_dataset[0])

{'goal': "When boiling butter, when it's ready, you can", 'sol1': 'Pour it onto a plate', 'sol2': 'Pour it into a jar', 'label': 1}
{'goal': "How do I ready a guinea pig cage for it's new occupants?", 'sol1': 'Provide the guinea pig with a cage full of a few inches of bedding made of ripped paper strips, you will also need to supply it with a water bottle and a food dish.', 'sol2': 'Provide the guinea pig with a cage full of a few inches of bedding made of ripped jeans material, you will also need to supply it with a water bottle and a food dish.', 'label': 0}
{'goal': 'how do you puncture a vein?', 'sol1': 'hit it at the wrong angle and make it bleed.', 'sol2': 'pop it.', 'label': -1}


In [11]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [37]:
from transformers import AutoModelForSequenceClassification

model_checkpoint = "bert-large-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
train_dataset = dataset['train']


In [14]:
print(train_dataset[1])

{'goal': 'To permanently attach metal legs to a chair, you can', 'sol1': 'Weld the metal together to get it to stay firmly in place', 'sol2': 'Nail the metal together to get it to stay firmly in place', 'label': 0}


In [42]:
print(len(labels))

32226


In [38]:
from torch.utils.data import Dataset

class PIQADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Assuming you're working with the training dataset
train_dataset = dataset['train']  # Accessing the 'train' split

# Encode the inputs for both solutions
encodings = tokenizer(
    [f"{item['goal']} [SEP] {item['sol1'] if item['label'] == 1 else item['sol2']}" 
     for item in train_dataset],
    truncation=True, padding=True, return_tensors='pt'
)

# Create the labels (1 for sol1, 0 for sol2)
labels = [item['label'] for item in train_dataset]

# Create the dataset
piqa_dataset = PIQADataset(encodings, labels)


In [39]:
print(encodings)

# Get the first sequence of input IDs
input_ids = encodings['input_ids'][0]

# Decode the input IDs back to text
decoded_text = tokenizer.decode(input_ids)

print(decoded_text)

{'input_ids': tensor([[  101,  2043, 16018,  ...,     0,     0,     0],
        [  101,  2000,  8642,  ...,     0,     0,     0],
        [  101,  2129,  2079,  ...,     0,     0,     0],
        ...,
        [  101,  2000,  7744,  ...,     0,     0,     0],
        [  101,  2191, 14380,  ...,     0,     0,     0],
        [  101,  2129,  2079,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
[CLS] when boiling butter, when it's ready, you can [SEP] pour it onto a plate [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [50]:
piqa_dataset[1]

{'input_ids': tensor([  101,  2000,  8642, 22476,  3384,  3456,  2000,  1037,  3242,  1010,
          2017,  2064,   102, 13774,  1996,  3384,  2362,  2000,  2131,  2009,
          2000,  2994,  7933,  1999,  2173,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [40]:
# Access the validation split
val_dataset = dataset['validation']

# Encode the inputs for the validation set
val_encodings = tokenizer(
    [f"{item['goal']} [SEP] {item['sol1'] if item['label'] == 1 else item['sol2']}" 
     for item in val_dataset],
    truncation=True, padding=True, return_tensors='pt'
)

# Extract the correct labels from the validation dataset
val_labels = [item['label'] for item in val_dataset]

# Create the validation dataset
val_piqadataset = PIQADataset(val_encodings, val_labels)

In [41]:
print(val_piqadataset[0])

{'input_ids': tensor([  101,  2129,  2079,  1045,  3201,  1037,  7102, 10369,  7980,  2005,
         2009,  1005,  1055,  2047, 18837,  1029,   102,  3073,  1996,  7102,
        10369,  2007,  1037,  7980,  2440,  1997,  1037,  2261,  5282,  1997,
         2793,  4667,  2081,  1997,  9157,  6312,  3430,  1010,  2017,  2097,
         2036,  2342,  2000,  4425,  2009,  2007,  1037,  2300,  5835,  1998,
         1037,  2833,  9841,  1012,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [42]:
from torch.utils.data import DataLoader

# Create the DataLoader
batch_size = 16  # You can adjust this based on your GPU's memory capacity
train_dataloader = DataLoader(piqa_dataset, batch_size=batch_size, shuffle=True)


In [43]:
from transformers import AutoModelForSequenceClassification

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="/kaggle/working/results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',            # Directory for storing logs
    report_to="none",                 # Disable W&B integration
    fp16=True,
)



/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [45]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=piqa_dataset,        # Your training dataset
    eval_dataset=val_piqadataset,      # Your validation dataset
    tokenizer=tokenizer
)



In [46]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss


In [26]:
import torch

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
model = model.to(torch.device("cpu"))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
device = torch.device("cpu")
model = model.to(device)

In [29]:
import torch
from tqdm import tqdm  # for progress bar

device = next(model.parameters()).device
print(f"Model is on device: {device}")

model.eval()
predictions = []

# Use tqdm for progress bar
with torch.no_grad():
    for i in tqdm(range(0, len(val_piqadataset), batch_size), desc="Processing batches"):
        # Ensure we don't go out of bounds
        batch_end = min(i + batch_size, len(val_piqadataset))
        batch = {k: v[i:batch_end].to(device) for k, v in val_encodings.items()}
        outputs = model(**batch)
        predictions.extend(outputs.logits.argmax(dim=-1).cpu().tolist())

# Process predictions
final_predictions = predictions  # For validation set, we don't need to pair the predictions

# Compare with true labels
true_labels = val_labels  # Using val_labels from the image
correct = sum(p == l for p, l in zip(final_predictions, true_labels))
accuracy = correct / len(true_labels)
print(f"Validation Accuracy: {accuracy:.4f}")

Model is on device: cpu


Processing batches: 100%|██████████| 115/115 [04:31<00:00,  2.36s/it]

Validation Accuracy: 0.5060


In [31]:
for i in range(5):  # Look at the first 5 examples
    goal = val_dataset['goal'][i]
    sol1 = val_dataset['sol1'][i]
    sol2 = val_dataset['sol2'][i]
    true_label = val_labels[i]  # Using val_labels as defined in the image
    predicted_label = final_predictions[i]
    
    print(f"Goal: {goal}")
    print(f"Solution 1: {sol1}")
    print(f"Solution 2: {sol2}")
    print(f"True label: {true_label}")
    print(f"Predicted label: {predicted_label}")
    print(f"Correct: {true_label == predicted_label}")
    print("\n")

Goal: How do I ready a guinea pig cage for it's new occupants?
Solution 1: Provide the guinea pig with a cage full of a few inches of bedding made of ripped paper strips, you will also need to supply it with a water bottle and a food dish.
Solution 2: Provide the guinea pig with a cage full of a few inches of bedding made of ripped jeans material, you will also need to supply it with a water bottle and a food dish.
True label: 0
Predicted label: 1
Correct: False


Goal: dresser
Solution 1: replace drawer with bobby pin 
Solution 2: finish, woodgrain with  bobby pin 
True label: 1
Predicted label: 1
Correct: True


Goal: To fight Ivan Drago in Rocky for sega master system.
Solution 1: Drago isn't in this game because it was released before Rocky IV.
Solution 2: You have to defeat Apollo Creed and Clubber Lang first.
True label: 1
Predicted label: 1
Correct: True


Goal: Make outdoor pillow.
Solution 1: Blow into tin can and tie with rubber band.
Solution 2: Blow into trash bag and tie w

In [32]:
# Initialize counters
correct_count = 0
wrong_count = 0

# Iterate through all examples in the validation dataset
for i in range(len(val_dataset)):
    true_label = val_labels[i]
    predicted_label = final_predictions[i]
    
    if true_label == predicted_label:
        correct_count += 1
    else:
        wrong_count += 1
    
    # Optional: Print details for the first 5 examples
    if i < 5:
        goal = val_dataset['goal'][i]
        sol1 = val_dataset['sol1'][i]
        sol2 = val_dataset['sol2'][i]
        
        print(f"Example {i+1}:")
        print(f"Goal: {goal}")
        print(f"Solution 1: {sol1}")
        print(f"Solution 2: {sol2}")
        print(f"True label: {true_label}")
        print(f"Predicted label: {predicted_label}")
        print(f"Correct: {true_label == predicted_label}")
        print("\n")

# Calculate accuracy
total_examples = len(val_dataset)
accuracy = correct_count / total_examples

# Print summary
print(f"Total examples: {total_examples}")
print(f"Correct predictions: {correct_count}")
print(f"Wrong predictions: {wrong_count}")
print(f"Accuracy: {accuracy:.4f}")

Example 1:
Goal: How do I ready a guinea pig cage for it's new occupants?
Solution 1: Provide the guinea pig with a cage full of a few inches of bedding made of ripped paper strips, you will also need to supply it with a water bottle and a food dish.
Solution 2: Provide the guinea pig with a cage full of a few inches of bedding made of ripped jeans material, you will also need to supply it with a water bottle and a food dish.
True label: 0
Predicted label: 1
Correct: False


Example 2:
Goal: dresser
Solution 1: replace drawer with bobby pin 
Solution 2: finish, woodgrain with  bobby pin 
True label: 1
Predicted label: 1
Correct: True


Example 3:
Goal: To fight Ivan Drago in Rocky for sega master system.
Solution 1: Drago isn't in this game because it was released before Rocky IV.
Solution 2: You have to defeat Apollo Creed and Clubber Lang first.
True label: 1
Predicted label: 1
Correct: True


Example 4:
Goal: Make outdoor pillow.
Solution 1: Blow into tin can and tie with rubber ban

In [36]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

def calculate_metrics(true_labels, predictions):
    # Accuracy
    accuracy = accuracy_score(true_labels, predictions)
    
    # Precision, Recall, and F1 Score
    precision = precision_score(true_labels, predictions, average='binary')
    recall = recall_score(true_labels, predictions, average='binary')
    f1 = f1_score(true_labels, predictions, average='binary')
    
    # Confusion Matrix
    tn, fp, fn, tp = confusion_matrix(true_labels, predictions).ravel()
    
    # Specificity (True Negative Rate)
    specificity = tn / (tn + fp)
    
    # Matthews Correlation Coefficient (MCC)
    mcc_numerator = (tp * tn) - (fp * fn)
    mcc_denominator = np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    mcc = mcc_numerator / mcc_denominator if mcc_denominator != 0 else 0
    
    return {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "Specificity": specificity,
        "MCC": mcc,
        "True Positives": tp,
        "True Negatives": tn,
        "False Positives": fp,
        "False Negatives": fn
    }

# After making predictions
metrics = calculate_metrics(val_labels, predictions)

# Print the results
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

# Calculate class distribution
class_distribution = np.bincount(val_labels) / len(val_labels)
print("\nClass Distribution:")
for i, proportion in enumerate(class_distribution):
    print(f"Class {i}: {proportion:.2%}")

Accuracy: 0.5060
Precision: 0.5056
Recall: 0.9784
F1 Score: 0.6667
Specificity: 0.0242
MCC: 0.0088
True Positives: 908.0000
True Negatives: 22.0000
False Positives: 888.0000
False Negatives: 20.0000

Class Distribution:
Class 0: 49.51%
Class 1: 50.49%
